<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/sl_whisper_base_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-eutn5x6i
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-eutn5x6i
  Resolved https://github.com/huggingface/transformers to commit 238d2e3c44366aba9dc5c770c95475765a6725cb
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269614 sha256=397a894686286f22610e9438208bab2f90be3f46a9152acfa6a3c8e340a1249b
  Stored in directory: /tmp/pip-ephem-wheel-cache-spqii9is/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [4]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "sl", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [5]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 309
})

In [6]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 27.5 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=c5d51e90ac63ef7de5e12532273e73312a009e8f9ee0d6e1b38bf61180a45077
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [47]:
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 75.5MiB/s]


In [48]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [49]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [50]:
text

,0
0,seveda lahko čestitamo za hiter razvoj elektro...
1,anžetu logarju in njegovim staršem je s smrtjo...
2,gre pa za to da imajo priložnost biti aktivni ...
3,zato je pomembno da evropska zveza kot sila mi...
4,vlado republike slovenije pozivam naj čim prej...
...,...
304,ta rešitev o kateri govorimo pa bo žal nezados...
305,sem za ukrepanje na področju podnebnih spremem...
306,sama ji nisem zelo naklonjena ker jo lahko raz...
307,beneška komisija je bila pri svojih zapisih ze...


In [51]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [52]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000
306,female,sama ji nisem zelo naklonjena ker jo lahko raz...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00250244140625, -0.001983642578125, -0.002...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [53]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,seveda lahko čestitamo za hiter razvoj elektro...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004638671875, 0.000244140625, -0.0036010742...",16000
1,male,anžetu logarju in njegovim staršem je s smrtjo...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0118408203125, -0.013702392578125, -0.0115...",16000
2,male,gre pa za to da imajo priložnost biti aktivni ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002410888671875, 0.006072998046875, 0.00354...",16000
3,male,zato je pomembno da evropska zveza kot sila mi...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.02227783203125, -0.00787353515625, 0.02120...",16000
4,male,vlado republike slovenije pozivam naj čim prej...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00048828125, -0.0001220703125, -0.001007080...",16000
...,...,...,...,...,...
297,male,nekateri seveda postavljajo tukaj dilemo ali n...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.1724853515625, -0.188323974609375, -0.1896...",16000
302,male,sistem vračanja iz grčije v turčijo ne deluje ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00018310546875, 0.000946044921875, 9.155273...",16000
303,male,od komisije pričakujemo da bo predložila smise...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0035400390625, -0.00372314453125, -0.00039...",16000
307,male,beneška komisija je bila pri svojih zapisih ze...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001373291015625, 9.1552734375e-05, -0.00161...",16000


In [54]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
7,female,na nekaj pa bi rada opozorila tukaj v svojem g...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, -0.003326416015625, -0.004821...",16000
12,female,iz tega razloga je tudi napačno sklepati da la...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.044647216796875, -0.03350830078125, -0.019...",16000
18,female,gre namreč za ugled nas vseh gre za ugled evro...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.03363037109375, -0.033935546875, -0.036132...",16000
20,female,z vsem spoštovanjem do novega organa in do izj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.1650390625, 0.13531494140625, 0.11520385742...",16000
26,female,v naši energetski mešanici je vse več premoga.,/root/.cache/huggingface/datasets/downloads/ex...,"[0.003143310546875, 0.00390625, 6.103515625e-0...",16000
...,...,...,...,...,...
300,female,moram reči da sem po dolgem času zadovoljna z ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.004669189453125, 0.012481689453125, 0.00991...",16000
301,female,napačna je namreč razlaga da je kategorija ope...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.024261474609375, 0.028076171875, 0.03970336...",16000
304,female,ta rešitev o kateri govorimo pa bo žal nezados...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000335693359375, -0.00262451171875, 0.01065...",16000
305,female,sem za ukrepanje na področju podnebnih spremem...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0211181640625, -0.003387451171875, 0.00769...",16000


In [55]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="sl", fp16=False)
    return result["text"]

In [56]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [57]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [58]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.1363636363636365, Precision: 0.2, Recall: 0.22727272727272727, F1: 0.21276595744680854
Processed 2 files, WER: 0.8461538461538461, Precision: 0.23076923076923078, Recall: 0.23076923076923078, F1: 0.23076923076923078
Processed 3 files, WER: 0.5333333333333333, Precision: 0.5, Recall: 0.4666666666666667, F1: 0.4827586206896552
Processed 4 files, WER: 1.0, Precision: 0.25, Recall: 0.21739130434782608, F1: 0.23255813953488372
Processed 5 files, WER: 0.6875, Precision: 0.39285714285714285, Recall: 0.3793103448275862, F1: 0.3859649122807017
Processed 6 files, WER: 0.8536585365853658, Precision: 0.3684210526315789, Recall: 0.4, F1: 0.3835616438356164
Processed 7 files, WER: 1.1333333333333333, Precision: 0.23529411764705882, Recall: 0.26666666666666666, F1: 0.25
Processed 8 files, WER: 0.7777777777777778, Precision: 0.3333333333333333, Recall: 0.23529411764705882, F1: 0.27586206896551724
Processed 9 files, WER: 0.7692307692307693, Precision: 0.3, Recall: 0.25, F1: 0.

In [59]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.7407407407407407, Precision: 0.46938775510204084, Recall: 0.35384615384615387, F1: 0.4035087719298246
Processed 2 files, WER: 1.0, Precision: 0.21428571428571427, Recall: 0.2, F1: 0.20689655172413796
Processed 3 files, WER: 0.45454545454545453, Precision: 0.5, Recall: 0.625, F1: 0.5555555555555556
Processed 4 files, WER: 0.8478260869565217, Precision: 0.27906976744186046, Recall: 0.3, F1: 0.2891566265060241
Processed 5 files, WER: 0.875, Precision: 0.16666666666666666, Recall: 0.125, F1: 0.14285714285714288
Processed 6 files, WER: 0.6190476190476191, Precision: 0.45, Recall: 0.42857142857142855, F1: 0.4390243902439024
Processed 7 files, WER: 0.5357142857142857, Precision: 0.5357142857142857, Recall: 0.5357142857142857, F1: 0.5357142857142857
Processed 8 files, WER: 0.68, Precision: 0.48, Recall: 0.5, F1: 0.4897959183673469
Processed 9 files, WER: 0.8947368421052632, Precision: 0.23529411764705882, Recall: 0.21052631578947367, F1: 0.2222222222222222
Processed 1